In [1]:
import pandas as pd

# text processing
import re

In [2]:
df = pd.read_csv('../Data/full_model_spec_sales_df_cleaned.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4092 entries, 0 to 4091
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       4092 non-null   int64  
 1   Total_Sales      4092 non-null   float64
 2   Year             4092 non-null   int64  
 3   cleaned_name     4092 non-null   object 
 4   curb_weight_lbs  2116 non-null   float64
 5   speed_sec        3142 non-null   float64
 6   horsepower_hp    3408 non-null   float64
 7   width_in         3052 non-null   float64
 8   msrp             3526 non-null   float64
 9   pass_capacity    3253 non-null   float64
 10  doors            3253 non-null   float64
 11  wheelbase_in     3539 non-null   float64
 12  height_in        3320 non-null   float64
 13  tank_cap_gal     3525 non-null   float64
 14  length_in        3017 non-null   float64
 15  vol_cubft        2428 non-null   float64
 16  mpg_comb         3539 non-null   float64
 17  drivetrain    

### load additonal datset

In [3]:
specs_df = pd.read_csv('/Users/racheldilley/Documents/car-sales-predictor-data/cars.csv')
specs_df['Year'] = specs_df['Identification.Year']
specs_df.drop(['Engine Information.Engine Type', 'Fuel Information.City mpg', 'Fuel Information.Highway mpg',
              'Identification.ID', 'Identification.Make', 'Identification.Year', 
               'Engine Information.Number of Forward Gears', 'Engine Information.Transmission',
              'Engine Information.Hybrid'], axis=1, inplace=True)
specs_df.drop_duplicates(['Identification.Model Year'], keep='first', inplace=True)
specs_df.head()

,Dimensions.Height,Dimensions.Length,Dimensions.Width,Engine Information.Driveline,Fuel Information.Fuel Type,Identification.Classification,Identification.Model Year,Engine Information.Engine Statistics.Horsepower,Engine Information.Engine Statistics.Torque,Year
0,140,143,202,All-wheel drive,Gasoline,Automatic transmission,2009 Audi A3,250,236,2009
5,91,17,62,All-wheel drive,Gasoline,Manual transmission,2009 Audi A5,265,243,2009
7,201,221,191,All-wheel drive,Gasoline,Automatic transmission,2009 Audi Q7,350,325,2009
9,147,96,34,All-wheel drive,Gasoline,Manual transmission,2009 Audi A4 Sedan,211,258,2009
12,172,63,87,Front-wheel drive,Gasoline,Automatic transmission,2012 Acura TL,280,254,2012


In [4]:
def clean_name(name):
    name = name[4:]
    name = name.lower()
    name = re.sub(r'[^\w\s]', ' ', name)
   
    return name.strip()

In [5]:
specs_df['cleaned_name'] = specs_df.apply(lambda x: clean_name(x['Identification.Model Year']), axis=1)


df2 = df.merge(specs_df, how = 'left',on=["cleaned_name", 'Year'])
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4093 entries, 0 to 4092
Data columns (total 30 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   Unnamed: 0                                       4093 non-null   int64  
 1   Total_Sales                                      4093 non-null   float64
 2   Year                                             4093 non-null   int64  
 3   cleaned_name                                     4093 non-null   object 
 4   curb_weight_lbs                                  2116 non-null   float64
 5   speed_sec                                        3143 non-null   float64
 6   horsepower_hp                                    3409 non-null   float64
 7   width_in                                         3053 non-null   float64
 8   msrp                                             3527 non-null   float64
 9   pass_capacity                 

In [6]:
import math
def fill_null_values(x, y):
    try:
        if math.isnan(x):
            return y
        else:
            return x
    except:
        return x

drop_columns = ['Identification.Model Year']
df3 = df2.copy()
df3.drop(drop_columns, axis=1, inplace=True )

column_pairs = {'heightIN': ['height_in','Dimensions.Height'], 'lengthIN': ['length_in', 'Dimensions.Length'], 
               'widthIN': ['width_in', 'Dimensions.Width'], 'drive': ['drivetrain', 'Engine Information.Driveline'],
               'trans': ['transmission', 'Identification.Classification'], 'fuel_type': ['fuel', 'Fuel Information.Fuel Type'],
                'horsepowerHP': ['horsepower_hp', 'Engine Information.Engine Statistics.Horsepower'] }

for key, value in column_pairs.items():
    df3[key] = df3.apply(lambda x: fill_null_values(x[value[0]], x[value[1]]), axis=1)
    df3.drop(value, axis=1, inplace=True)

df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4093 entries, 0 to 4092
Data columns (total 22 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   Unnamed: 0                                   4093 non-null   int64  
 1   Total_Sales                                  4093 non-null   float64
 2   Year                                         4093 non-null   int64  
 3   cleaned_name                                 4093 non-null   object 
 4   curb_weight_lbs                              2116 non-null   float64
 5   speed_sec                                    3143 non-null   float64
 6   msrp                                         3527 non-null   float64
 7   pass_capacity                                3254 non-null   float64
 8   doors                                        3254 non-null   float64
 9   wheelbase_in                                 3540 non-null   float64
 10  

In [7]:
df4 = df3.merge(specs_df, how = 'left',on=["cleaned_name"])
df4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7209 entries, 0 to 7208
Data columns (total 32 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   Unnamed: 0                                       7209 non-null   int64  
 1   Total_Sales                                      7209 non-null   float64
 2   Year_x                                           7209 non-null   int64  
 3   cleaned_name                                     7209 non-null   object 
 4   curb_weight_lbs                                  4027 non-null   float64
 5   speed_sec                                        5778 non-null   float64
 6   msrp                                             6429 non-null   float64
 7   pass_capacity                                    6021 non-null   float64
 8   doors                                            6021 non-null   float64
 9   wheelbase_in                  

In [8]:
drop_columns = ['Identification.Model Year', 'Year_y', 'Year_x', 'Unnamed: 0']
df5 = df4.copy()
df5['Year'] = df5['Year_x']
df5.drop(drop_columns, axis=1, inplace=True )

column_pairs = {'height_in': ['heightIN','Dimensions.Height'], 'length_in': ['lengthIN', 'Dimensions.Length'], 
               'width_in': ['widthIN', 'Dimensions.Width'], 'drivetrain': ['drive', 'Engine Information.Driveline'],
               'transmission': ['trans', 'Identification.Classification'], 'fuel': ['fuel_type', 'Fuel Information.Fuel Type'],
               'horsepower_hp': ['horsepowerHP', 'Engine Information.Engine Statistics.Horsepower'], 'torque':
                ['Engine Information.Engine Statistics.Torque_x', 'Engine Information.Engine Statistics.Torque_y'] }

for key, value in column_pairs.items():
    df5[key] = df5.apply(lambda x: fill_null_values(x[value[0]], x[value[1]]), axis=1)
    df5.drop(value, axis=1, inplace=True)

df5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7209 entries, 0 to 7208
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Total_Sales      7209 non-null   float64
 1   cleaned_name     7209 non-null   object 
 2   curb_weight_lbs  4027 non-null   float64
 3   speed_sec        5778 non-null   float64
 4   msrp             6429 non-null   float64
 5   pass_capacity    6021 non-null   float64
 6   doors            6021 non-null   float64
 7   wheelbase_in     6455 non-null   float64
 8   tank_cap_gal     6449 non-null   float64
 9   vol_cubft        4494 non-null   float64
 10  mpg_comb         6429 non-null   float64
 11  class            5160 non-null   object 
 12  Year             7209 non-null   int64  
 13  height_in        6646 non-null   float64
 14  length_in        6552 non-null   float64
 15  width_in         6625 non-null   float64
 16  drivetrain       6819 non-null   object 
 17  transmission  

In [9]:
df5.drop_duplicates(['cleaned_name', 'Year'], keep='first', inplace=True)

df5.drop(['torque'], axis=1, inplace=True)
df5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4092 entries, 0 to 7206
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Total_Sales      4092 non-null   float64
 1   cleaned_name     4092 non-null   object 
 2   curb_weight_lbs  2116 non-null   float64
 3   speed_sec        3142 non-null   float64
 4   msrp             3526 non-null   float64
 5   pass_capacity    3253 non-null   float64
 6   doors            3253 non-null   float64
 7   wheelbase_in     3539 non-null   float64
 8   tank_cap_gal     3525 non-null   float64
 9   vol_cubft        2428 non-null   float64
 10  mpg_comb         3539 non-null   float64
 11  class            2831 non-null   object 
 12  Year             4092 non-null   int64  
 13  height_in        3529 non-null   float64
 14  length_in        3435 non-null   float64
 15  width_in         3508 non-null   float64
 16  drivetrain       3702 non-null   object 
 17  transmission  

In [10]:
#save as csv to manually add in missing data
df5.to_csv('../Data/full_model_spec_sales_df_cleaned2.csv')

In [18]:
df6 = pd.read_csv('../Data/full_model_spec_sales_df_cleaned3.csv')

In [19]:
null_cols = ['height_in', 'length_in', 'width_in', 'drivetrain', 'transmission', 'fuel', 'class', 'horsepower_hp']
null_df = df6[df6[null_cols].isna().all(1)] #.groupby('cleaned_name')
null_df['cleaned_name'].value_counts()[:15]
# null_weight = df5[df5[null_cols].isnull()]
# null_weight

chevrolet caprice ppv          5
mazda b                        5
ford c max                     5
lexus ct200h                   5
suzuki forenza reno            5
volkswagen golf rabbit         5
chevrolet bolt                 5
audi a4 allroad                5
dodge caravan grand caravan    4
ford transit                   4
ford e                         4
lexus lx570                    4
buick rainier                  4
isuzu i 280 i 350              4
lexus hs250h                   4
Name: cleaned_name, dtype: int64

In [17]:
df6.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4092 entries, 0 to 4091
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       4092 non-null   int64  
 1   Total_Sales      4092 non-null   float64
 2   cleaned_name     4092 non-null   object 
 3   curb_weight_lbs  2163 non-null   float64
 4   speed_sec        3180 non-null   float64
 5   msrp             3565 non-null   object 
 6   pass_capacity    3297 non-null   float64
 7   doors            3297 non-null   float64
 8   wheelbase_in     3577 non-null   float64
 9   tank_cap_gal     3563 non-null   float64
 10  vol_cubft        2470 non-null   float64
 11  mpg_comb         3577 non-null   float64
 12  class            2873 non-null   object 
 13  Year             4092 non-null   int64  
 14  height_in        3567 non-null   float64
 15  length_in        3473 non-null   float64
 16  width_in         3546 non-null   float64
 17  drivetrain    